/**
* @par Copyright (C): 2010-2020, Shenzhen Yahboom Tech
* @file:         Ultrasonic+IR avoid.ipynb
* @author:       xiaozhen
* @version：     V1.0
* @date:         2020.10.11
* @brief:        .Ultrasonic+IR avoid function
* @details:
* @par History:  Description below
*/

### Import library

In [11]:
#-*- coding:UTF-8 -*-
import RPi.GPIO as GPIO
import time
import YB_Pcb_Car   

car = YB_Pcb_Car.YB_Pcb_Car()

### Define the pins used in this course.
### Set the coding method of GPIO and initial settings.

In [12]:
#Set the GPIO port to BIARD encoding mode
GPIO.setmode(GPIO.BOARD)

#Ignore the warning message
GPIO.setwarnings(False)

AvoidSensorLeft = 21     #Left infrared obstacle avoidance sensor pin
AvoidSensorRight = 19    #Right infrared obstacle avoidance sensor pin
Avoid_ON = 22   #Infrared obstacle avoidance sensor switch pin

#Define the pins of the ultrasonic module
EchoPin = 18
TrigPin = 16

#Set pin mode
GPIO.setup(AvoidSensorLeft,GPIO.IN)
GPIO.setup(AvoidSensorRight,GPIO.IN)
GPIO.setup(Avoid_ON,GPIO.OUT)
GPIO.setup(EchoPin,GPIO.IN)
GPIO.setup(TrigPin,GPIO.OUT)
GPIO.output(Avoid_ON,GPIO.HIGH)

In [13]:
#Ultrasonic function
def Distance():
    GPIO.output(TrigPin,GPIO.LOW)
    time.sleep(0.000002)
    GPIO.output(TrigPin,GPIO.HIGH)
    time.sleep(0.000015)
    GPIO.output(TrigPin,GPIO.LOW)

    t3 = time.time()

    while not GPIO.input(EchoPin):
        t4 = time.time()
        if (t4 - t3) > 0.03 :
            return -1
    t1 = time.time()
    while GPIO.input(EchoPin):
        t5 = time.time()
        if(t5 - t1) > 0.03 :
            return -1

    t2 = time.time()
    #time.sleep(0.01)
    #print ("distance_1 is %d " % (((t2 - t1)* 340 / 2) * 100))
    return ((t2 - t1)* 340 / 2) * 100


In [14]:
def Distance_test():
    num = 0
    ultrasonic = []
    while num < 5:
            distance = Distance()
            #print("distance is %f"%(distance) )
            while int(distance) == -1 :
                distance = Distance()
                #print("Tdistance is %f"%(distance) )
            while (int(distance) >= 500 or int(distance) == 0) :
                distance = Distance()
                #print("Edistance is %f"%(distance) )
            ultrasonic.append(distance)
            num = num + 1
            #time.sleep(0.01)
    #print ('ultrasonic')
    distance = (ultrasonic[1] + ultrasonic[2] + ultrasonic[3])/3
    #print("distance is %f"%(distance) ) 
    return distance

### Avoid function is used to complete the infrared obstacle avoidance function of the car

In [15]:
import cv2
import ipywidgets.widgets as widgets
import threading
import time

image_widget = widgets.Image(format='jpeg', width=600, height=500)  
display(image_widget)  

Image(value=b'', format='jpeg', height='500', width='600')

In [16]:
def avoid():
    distance = Distance_test()
    # modified
    LeftSensorValue  =  GPIO.input(AvoidSensorLeft);
    RightSensorValue =  GPIO.input(AvoidSensorRight);
    print(f"right value : {RightSensorValue}, left value : {LeftSensorValue}, distance : {distance}")
   
    
    #With obstacle pin is low level, the indicator light is on, without obstacle, pin is high level, the indicator light is off
    if distance < 15 and LeftSensorValue == False and RightSensorValue == False :
        car.Car_Stop() 
        time.sleep(0.1)
        car.Car_Spin_Right(100,100) 
        time.sleep(1)
    elif distance < 15 and LeftSensorValue == True and RightSensorValue == False :
        car.Car_Stop()
        time.sleep(0.1)
        car.Car_Spin_Left(80,80) 
        time.sleep(1)
        if LeftSensorValue == False and RightSensorValue == True :
            car.Car_Stop()
            time.sleep(0.1)
            car.Car_Spin_Right(90,90) 
            time.sleep(2)
    elif distance < 15 and LeftSensorValue == False and RightSensorValue == True :
        car.Car_Stop() 
        time.sleep(0.1)
        car.Car_Spin_Right(80,80)
        time.sleep(1)
        if LeftSensorValue == True and RightSensorValue == False  :
            car.Car_Stop()
            time.sleep(0.1)
            car.Car_Spin_Left(90,90) 
            time.sleep(2)
    elif distance < 15 and LeftSensorValue == True and RightSensorValue == True :
        car.Car_Stop() 
        time.sleep(0.1)
        car.Car_Spin_Right(80,80) 
        time.sleep(0.5)
    elif distance >= 15 and LeftSensorValue == False and RightSensorValue == False :
        car.Car_Stop() 
        time.sleep(0.1)
        car.Car_Spin_Right(90,90) 
        time.sleep(1)
    elif distance >= 15 and LeftSensorValue == False and RightSensorValue == True :
        car.Car_Stop() 
        time.sleep(0.1)
        car.Car_Spin_Right(80,80) 
        time.sleep(0.5)
    elif distance >= 15 and LeftSensorValue == True and RightSensorValue == False : 
        car.Car_Stop() 
        time.sleep(0.1)
        car.Car_Spin_Left(80,80) 
        time.sleep(0.5)
    else:
        car.Car_Run(100,100) 
    

    print(f"right value : {RightSensorValue}, left value : {LeftSensorValue}, distance : {distance}")
   
   

In [17]:
def run():
    distance = Distance_test()
    if(distance < 20):
        car.Car_Stop()
        time.sleep(0.1)
        choice()
        #time.sleep(1)
    else:
        modify();
        #car.Car_Stop()
        #time.sleep(0.1)
        #car.Car_Run(50,50)
        #time.sleep(1)
        print(f"distance = {distance}")

In [18]:
def choice():
    LeftSensorValue  =  GPIO.input(AvoidSensorLeft)
    RightSensorValue =  GPIO.input(AvoidSensorRight)
   
    #print(f"right value : {RightSensorValue}, left value : {LeftSensorValue}")
    print("choice")
    car.Car_Spin_Left(160, 160)
    time.sleep(0.3)
    car.Car_Stop()
    distance = Distance_test()
    if(distance < 20):
        print("right")
        car.Car_Spin_Right(160, 160)
        time.sleep(0.6)
        car.Car_Stop()
        time.sleep(0.1)
    
    

In [19]:
def modify():
    LeftSensorValue  =  GPIO.input(AvoidSensorLeft);
    RightSensorValue =  GPIO.input(AvoidSensorRight);
    print("modi")
    if LeftSensorValue == True and RightSensorValue == False :
        car.Car_Left(0,100) 
        time.sleep(0.2)
        car.Car_Stop()
       
        #time.sleep(0.5)
        #car.Car_Run(50,50)

    elif LeftSensorValue == False and RightSensorValue == True :
        car.Car_Right(100,0)
        time.sleep(0.2)
        car.Car_Stop() 
    else:  
        car.Car_Run(150,150)
        time.sleep(0.1)
        #car.Car_Run(50,50)
#     else:
#         car.Car_Run(50,50)

In [20]:
try:
    while True:
        run()
        #avoid()
except KeyboardInterrupt:
    pass
car.Car_Stop() 
del car
print("Ending")
GPIO.cleanup()

modi
distance = 368.82273356119794
modi
distance = 368.9889113108317
modi
distance = 40.27986526489258
modi
distance = 348.93274307250977
modi
distance = 35.822788874308266
modi
distance = 42.11052258809408
modi
distance = 98.9527702331543
modi
distance = 128.53376070658365
modi
distance = 129.07822926839194
modi
Ending


### When you need to end the entire program, please click the square button (Interrupt the kernel) in the upper menu bar.
### Then you can see the Ending prompt, which means the program has been successfully ended